In [13]:
import pandas as pd

In [14]:
df = pd.read_csv("./dataset/dataset_avaliacoes.csv")

In [ ]:
# Retorna as 5 primeiras linhas
df.head()

,ID_avaliacao,avaliacao,nota,sentimento
0,81841,"Esse smartphone superou minhas expectativas, r...",5,positivo
1,28031,o cheiro e muito desagradavel!! pra quem se in...,2,negativo
2,91332,"Eu fiz a compra a loja não entregou o produto,...",1,negativo
3,110045,"Atendeu minhas expectativas, só achei a luz ru...",4,positivo
4,36929,"Gostei muito, só achei que ficou a desejar a q...",5,positivo


In [ ]:
# Retorna número de linhas e quantidade de colunas
df.shape

(15501, 4)